In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from itertools import combinations

from processor.fdr import FDR_INFO_PROCESSOR, FDR_OHLCV_PREPROCESSOR
from processor.factor.technical import LINEAR_COEFFICENT_FACTOR_PROCESSOR
from processor.factor.parameter import FACTOR_PARAMETER_PROCESSOR

from analyser.factor import FACTOR_ANALYSER

In [2]:
CFG = {
    "analysis_date": "2023-11-01",
    
    "LINEAR_COEFFICENT_FACTOR_CFG": {
        "factors": ["Close", "VolumeRotation"],
        "moving_average_window": 10,
        "coef_recent_n": 30,
    },
    
    "FACTOR_ANALYSIS_CFG": {
        "profit_recent_n": 20,
    },
}

In [3]:
# fdr_info
fdr_info_df = pd.read_csv("./data/fdr_info_df.csv", index_col=0)
fdr_info_processor = FDR_INFO_PROCESSOR(fdr_info_df)

In [4]:
# fdr_ohlcv
fdr_ohlcv_df = pd.read_csv("./data/fdr_ohlcv_df.csv", index_col=0)

fdr_ohlcv_preprocessor = FDR_OHLCV_PREPROCESSOR(fdr_ohlcv_df)
pps_fdr_ohlcv_df = fdr_ohlcv_preprocessor.get_pps_fdr_ohlcv_df()

In [5]:
# split ohlcv

future_ohlcv_df = pps_fdr_ohlcv_df[CFG["analysis_date"] < pps_fdr_ohlcv_df["Date"]].copy()
ohlcv_df = pps_fdr_ohlcv_df[pps_fdr_ohlcv_df["Date"] < CFG["analysis_date"]].copy()

In [6]:
# get linear_coef_factors_df
linear_coefficent_factor_processor = LINEAR_COEFFICENT_FACTOR_PROCESSOR(ohlcv_df, fdr_info_processor)
linear_coef_factors_df = linear_coefficent_factor_processor.get_linear_coef_factor_df(
    CFG["LINEAR_COEFFICENT_FACTOR_CFG"]
)

In [7]:
# Analysis
factor_analyser = FACTOR_ANALYSER(linear_coef_factors_df)

In [8]:
factors = [col for col in linear_coef_factors_df.columns if col != "StockCode"]
factor_combs = list(combinations(factors, 2))

In [9]:
params_dict = dict()

for factor_comb in factor_combs:
    profit_analysis_2d_df = factor_analyser.get_profit_analysis_2d_df(
        future_ohlcv_df, CFG["FACTOR_ANALYSIS_CFG"], factor_comb
    )
    profit_analysis_2d_df.fillna(profit_analysis_2d_df.mean().mean(), inplace=True)

    factor_parameter_processor = FACTOR_PARAMETER_PROCESSOR(profit_analysis_2d_df, 3, 3)
    best_args = factor_parameter_processor.get_best_args()
    best_value = factor_parameter_processor.get_best_value()
    params_dict[best_args] = best_value

In [10]:
params_df = pd.DataFrame().from_dict(params_dict, orient="index",columns=['Value'])

In [11]:
params_df.nlargest(5, "Value")

,Value
"((Close, 0.5, 0.8), (VolumeRotation, 0.1, 0.4))",0.085237


In [12]:
list(params_df.nlargest(5, "Value").index)

[(('Close', 0.5, 0.8), ('VolumeRotation', 0.1, 0.4))]